In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import networkx as nx

In [2]:
import tools

In [3]:
data = pd.read_json('data/flink/reviews_20220602-16h22m23s_apache_flink_master_pr-reviews.txt', lines=True)
tools.initialize_datetime(data)

In [4]:
data = data.sort_values(by='createdAt').reset_index(drop=True)

In [5]:
data

,createdAt,publishedAt,reviewerLogin,authorLogin,number
0,2016-09-15 16:18:26+00:00,2016-09-15 16:18:26+00:00,ggevay,ramkrish86,2496
1,2016-09-15 16:28:14+00:00,2016-09-15 16:28:14+00:00,ggevay,ramkrish86,2496
2,2016-09-15 16:31:19+00:00,2016-09-15 16:31:19+00:00,ramkrish86,ramkrish86,2496
3,2016-09-15 16:35:50+00:00,2016-09-15 16:35:50+00:00,ramkrish86,ramkrish86,2496
4,2016-09-15 16:49:18+00:00,2016-09-15 16:49:18+00:00,ggevay,ramkrish86,2496
...,...,...,...,...,...
65830,2022-06-02 12:05:55+00:00,2022-06-02 12:05:55+00:00,wuchong,liuyongvs,19746
65831,2022-06-02 12:21:21+00:00,2022-06-02 12:21:21+00:00,echauchot,echauchot,19680
65832,2022-06-02 12:45:27+00:00,2022-06-02 12:45:27+00:00,wuchong,luoyuxia,18958
65833,2022-06-02 14:17:53+00:00,2022-06-02 14:17:53+00:00,luoyuxia,luoyuxia,18958


In [6]:
reviewer_is_flinkbot = data['reviewerLogin'] == 'flinkbot'
reviewer_is_author = data['reviewerLogin'] == data['authorLogin']

In [7]:
review_data = data.loc[~reviewer_is_author & ~reviewer_is_flinkbot].sort_values(by='createdAt')

In [8]:
review_data = review_data.loc[review_data[['authorLogin', 'reviewerLogin', 'number']].drop_duplicates().index]
review_data = review_data.drop(columns='publishedAt')
review_data = review_data.reset_index(drop=True)

In [9]:
review_data

,createdAt,reviewerLogin,authorLogin,number
0,2016-09-15 16:18:26+00:00,ggevay,ramkrish86,2496
1,2016-09-16 11:34:11+00:00,mxm,vijikarthi,2275
2,2016-09-16 12:16:19+00:00,mxm,alkagin,2149
3,2016-09-19 11:21:34+00:00,ggevay,ramkrish86,2510
4,2016-09-19 13:30:47+00:00,fhueske,mushketyk,2430
...,...,...,...,...
15098,2022-06-01 08:05:33+00:00,wuchong,liuyongvs,19746
15099,2022-06-01 12:48:50+00:00,RyanSkraba,zentol,19841
15100,2022-06-01 15:00:40+00:00,Myasuka,zoltar9264,19789
15101,2022-06-02 02:07:12+00:00,HuangXingBo,deadwind4,19802


In [11]:
review_data_count = review_data[['authorLogin', 'reviewerLogin']].value_counts().reset_index().rename(columns={0: 'count'})

In [12]:
review_data_count.shape

(4417, 3)

In [16]:
review_data_count_above = review_data_count.loc[review_data_count['count'] > 10]

In [17]:
review_data_count.rename(columns={'reviewerLogin': 'source', 'authorLogin': 'target', 'count': 'weight'}).to_csv('output/aggregated_review_graph.csv', index=False)

In [18]:
review_data.rename(columns={'reviewerLogin': 'source', 'authorLogin': 'target', 'date': 'createdAt'}).to_csv('output/aggregated_review_graph.csv', index=False)

In [22]:
top_reviewers = review_data_count.groupby('reviewerLogin')['count'].sum().sort_values(ascending=False)
tools.writefile(top_reviewers, "top_reviewers")
with pd.option_context("display.max_rows", 10000000):
    print(top_reviewers)

reviewerLogin
zentol                1357
tillrohrmann          1351
wuchong                922
twalthr                618
pnowojski              544
aljoscha               445
JingsongLi             439
dawidwys               380
dianfu                 377
KurtYoung              304
GJL                    280
fhueske                270
rmetzger               254
AHeise                 229
godfreyhe              219
kl0u                   215
StephanEwen            210
azagrebin              199
XComp                  197
tisonkun               185
tzulitai               174
xuefuz                 167
StefanRRichter         167
Myasuka                166
hequn8128              164
lirui-apache           163
rkhachatryan           162
xintongsong            159
bowenli86              154
zhuzhurk               145
leonardBang            144
Airblader              143
sunjincheng121         140
MartijnVisser          139
fapaul                 137
klion26                131
zhijiangW     